# Data 1 | Clean Voterfiles

First I unzip the text files, open the text files as dataframe, the delete the textfile. Then I clean the dataframe:
1. Generate new variables
2. Rename necessary variables
3. Drop unnecessary variables
4. Reformat the necessary variables
5. Then generate new variables using the reformatted variables

All data retrieved from https://dl.ncsbe.gov/?prefix=data/

## Merge Turnout Files

In [1]:
from datalink import *

hists = [x for x in os.listdir(path_0 + 'history') if 'ncvhis_Statewide' in x and '.txt' in x]
hists = [pd.read_csv(path_0 + 'history/'+x, sep='\t', header=0) for x in hists]
print('Loaded History Files')

hist = pd.concat(hists, ignore_index=True)
del hists
print('Concat Finished')

hist = hist.drop_duplicates()
print('Dropped Duplicates')

hist.to_pickle(path_0 + 'history/turnout_history.pkl')

## Create Clean Chunks

In [ ]:
from data1_clean import *

history = pd.read_pickle(f'{path_0}/history/turnout_history.pkl')

for date in dates:
    print(date)
    file, year = zip_map[date], year_map[date]
    
    if f'{date}_chunk_0.pkl' in os.listdir(path_1):
        print('  Done')
        
    if int(year) < 2010:
        print('  Too Old (for now)')
        
    if (f'{date}_chunk_0.pkl' not in os.listdir(path_1)) and (int(year) >= 2010):
        
        """ Unzip Snapshot """
        
        os.system(f'unzip -o {path_0_zip}{file} -d {path_0_zip}')
        zipfile = ZipFile(f'{path_0_zip}{file}')
        extracted_file, = zipfile.namelist()

        """ Chunk and Clean Snapshot """

        voter_chunks = pd.read_csv(path_0_zip + extracted_file, 
                                    low_memory=False, 
                                    iterator=True, 
                                    chunksize=10**6, 
                                    sep='\t', 
                                    encoding_errors='ignore', 
                                    error_bad_lines=False, 
                                    quoting=csv.QUOTE_NONE)
                                    
        for index, voters in enumerate(voter_chunks):
            print(f'  Cleaning Chunk {index}')
            voters = clean_NC(voters, history, date, nc_rename, run_elections=True)
            print(f'  Saving Chunk {index}')

            voters.to_pickle(f'{path_1}{date}_chunk_{index}.pkl')
        
        os.remove(path_0_zip + extracted_file)

In [ ]:
        """
        try:
            voter_chunks = pd.read_csv(path_0_zip + extracted_file, 
                                       low_memory=False, 
                                       iterator=True, 
                                       chunksize=10**6, 
                                       sep='\t', 
                                       encoding='utf-16', 
                                       encoding_errors='ignore', 
                                       error_bad_lines=False, 
                                       quoting=csv.QUOTE_NONE)
        except:
        print('Not encoding utf-16')
        """

## Trusty Old Version

This code works with previously extracted and converted csv files. The new version extracts the snapshot zip file.

In [4]:
from data1_clean import *

with open(path_0 + 'turnout_history_20190101.pkl','rb') as f: 
    history = pickle.load(f)
with open(path_0 + 'history_idus_20190101.pkl','rb') as f: 
    history_idus = pickle.load(f)

snapshot_filelist = sorted([x for x in os.listdir(path_0_csv) if x.split('.')[-1] == 'csv'])
for file in snapshot_filelist:
    print(file)
    
    if 'NC_' + year + '_chunk_0.pkl' in os.listdir(path_1):
        print('  Done')
        
    else:
        year = file.split('_')[-1][:4]

        voter_chunks, voter_list, chunk_index = pd.read_csv(path_0_csv + file, dtype='str', low_memory=False, iterator=True, chunksize=10**6), [], -1
        for voters in voter_chunks:

            print('  Cleaning Chunk ' + str(chunk_index))# + ' (of '+str(len(voter_chunks))+')')
            voters, chunk_index = clean_NC(voters, history, history_idus, year, nc_rename), chunk_index + 1
            print('  Saving Chunk ' + str(chunk_index))# + ' (of '+str(len(voter_chunks))+')')

            with open(path_1 + '/NC_' + year + '_chunk_' + str(chunk_index) + '.pkl','wb') as f:
                pickle.dump(voters, f)

VR_Snapshot_20051125.csv
  Done
VR_Snapshot_20060210.csv
  Done
VR_Snapshot_20070119.csv
  Done
VR_Snapshot_20081104.csv
  Done
VR_Snapshot_20090101.csv
  Done
VR_Snapshot_20100101.csv
  Done
VR_Snapshot_20110101.csv
  Done
VR_Snapshot_20120101.csv
  Done
VR_Snapshot_20130101.csv
  Done
VR_Snapshot_20150101.csv
  Done
VR_Snapshot_20160101.csv
  Done
VR_Snapshot_20161108.csv
  Done
VR_Snapshot_20170101.csv
  Done
VR_Snapshot_20180101.csv
  Done
VR_Snapshot_20190101.csv
  Done
